## Query: quantities + broader

In [30]:
from SPARQLWrapper import SPARQLWrapper, POST, BASIC, JSON

sparql = SPARQLWrapper("http://www.qudt.org/fuseki/qudt/sparql")
sparql.setMethod(POST)

sparql.setQuery("""
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?quantityKind ?broader 
WHERE {
	?quantityKind  rdf:type qudt:QuantityKind ;
	OPTIONAL { ?quantityKind skos:broader ?broaderRAW ; }
  	BIND (IF (BOUND(?broaderRAW),?broaderRAW, 'mainQ') AS ?broader)
} 
ORDER BY ASC (?quantityKind)
LIMIT 999
""")
sparql.setReturnFormat(JSON)
quantityKind = sparql.query().convert()

### set dictionaries of all quantities and split quantities with/without broader

In [31]:
allQuantities = {}
for idx in quantityKind['results']['bindings']:
    allQuantities[idx['quantityKind']['value']] = {}

mainQuantities = {}
broaderQuantities = {}

for idx in quantityKind['results']['bindings']:
    if(idx['broader']['value']=='mainQ'):
        mainQuantities[idx['quantityKind']['value']] = {}
    else: 
        broaderQuantities[idx['quantityKind']['value']] = idx['broader']['value']

print(len(mainQuantities.keys()))
print(len(broaderQuantities.keys()))

#for key,value in broaderQuantities.items():
#    print(key,'\nbelongsToBroader\n', value, '\n')



518
343


### check if every broader is part of all quantitiy dict
-> true

In [32]:
counter1 = 0
counter2 = 0
inAQ = {}
notinAQ = {}
for key,value in broaderQuantities.items():
    if value in allQuantities.keys():
        counter1 = counter1 +1
        inAQ[key] = {}
    if value not in allQuantities.keys():
        counter2 = counter2 +1
        notinAQ[key] = {}
        

print(counter1)
print(counter2)
print(notinAQ.keys())

343
0
dict_keys([])


### check if every child of broader is available in main quantity
-> more nested cascades of broader (notinMQ)

In [33]:
inMQ = {}
notinMQ = {}
for key,value in broaderQuantities.items():
    if value in mainQuantities.keys():
        inMQ[key] = value
    if value not in mainQuantities.keys():
        notinMQ[key] = value
        

print(len(inMQ.keys()))
print(len(notinMQ.keys()))
for k,v in inMQ.items():
    print(k, v)

224
119
http://qudt.org/vocab/quantitykind/AbsoluteActivity http://qudt.org/vocab/quantitykind/InverseVolume
http://qudt.org/vocab/quantitykind/AbsoluteHumidity http://qudt.org/vocab/quantitykind/Density
http://qudt.org/vocab/quantitykind/AbsorbedDose http://qudt.org/vocab/quantitykind/SpecificEnergy
http://qudt.org/vocab/quantitykind/AccelerationOfGravity http://qudt.org/vocab/quantitykind/Acceleration
http://qudt.org/vocab/quantitykind/ActiveEnergy http://qudt.org/vocab/quantitykind/Energy
http://qudt.org/vocab/quantitykind/AlphaDisintegrationEnergy http://qudt.org/vocab/quantitykind/Energy
http://qudt.org/vocab/quantitykind/Altitude http://qudt.org/vocab/quantitykind/Length
http://qudt.org/vocab/quantitykind/AmountOfSubstancePerUnitMass http://qudt.org/vocab/quantitykind/Concentration
http://qudt.org/vocab/quantitykind/AmountOfSubstancePerUnitVolume http://qudt.org/vocab/quantitykind/Concentration
http://qudt.org/vocab/quantitykind/AngularAcceleration http://qudt.org/vocab/quantityk

### check more nested cascades of broader

In [34]:
inMQ2 = {}
notinMQ2 = {}
for key,value in notinMQ.items():
    if value in inMQ.keys():
        inMQ2[key] = value
    if value not in inMQ.keys():
        notinMQ2[key] = value

print(len(inMQ2.keys()))
print(len(notinMQ2.keys()))

93
26


In [35]:
inMQ3 = {}
notinMQ3 = {}
for key,value in notinMQ2.items():
    if value in inMQ2.keys():
        inMQ3[key] = value
    if value not in inMQ2.keys():
        notinMQ3[key] = value

print(len(inMQ3.keys()))
print(len(notinMQ3.keys()))
for k,v in inMQ3.items():
    print(k,'\n', v,'\n')

25
1
http://qudt.org/vocab/quantitykind/ActivePower 
 http://qudt.org/vocab/quantitykind/ComplexPower 

http://qudt.org/vocab/quantitykind/Activity 
 http://qudt.org/vocab/quantitykind/StochasticProcess 

http://qudt.org/vocab/quantitykind/AngleOfAttack 
 http://qudt.org/vocab/quantitykind/Angle 

http://qudt.org/vocab/quantitykind/AngleOfOpticalRotation 
 http://qudt.org/vocab/quantitykind/Angle 

http://qudt.org/vocab/quantitykind/AngularDistance 
 http://qudt.org/vocab/quantitykind/Angle 

http://qudt.org/vocab/quantitykind/ApparentPower 
 http://qudt.org/vocab/quantitykind/ComplexPower 

http://qudt.org/vocab/quantitykind/AverageHeadEndPressure 
 http://qudt.org/vocab/quantitykind/HeadEndPressure 

http://qudt.org/vocab/quantitykind/AverageVacuumThrust 
 http://qudt.org/vocab/quantitykind/VacuumThrust 

http://qudt.org/vocab/quantitykind/BevelGearPitchAngle 
 http://qudt.org/vocab/quantitykind/Angle 

http://qudt.org/vocab/quantitykind/BraggAngle 
 http://qudt.org/vocab/quantitykin

In [36]:
inMQ4 = {}
notinMQ4 = {}
for key,value in notinMQ3.items():
    if value in inMQ3.keys():
        inMQ4[key] = value
    if value not in inMQ3.keys():
        notinMQ4[key] = value

print(len(inMQ4.keys()))
print(len(notinMQ4.keys()))
print(inMQ4.items())

1
0
dict_items([('http://qudt.org/vocab/quantitykind/RelativeHumidity', 'http://qudt.org/vocab/quantitykind/RelativePartialPressure')])


### Build sorted dictionary

In [43]:
q4 = {}
for k,v in inMQ4.items():
    if v not in q4.keys():
        q4[v] = {k: {'b4'}}
    else:
        q4[v].update({k: {'b4'}})

    
print(q4)

{'http://qudt.org/vocab/quantitykind/RelativePartialPressure': {'http://qudt.org/vocab/quantitykind/RelativeHumidity': {'b4'}}}


In [44]:
q3 = {}
for k,v in inMQ3.items():
    if v not in q3.keys():
        q3[v] = {k: {'b3'}}
    else:
        q3[v].update({k: {'b3'}})


# update q3 with q4 to be done ({'http://qudt.org/vocab/quantitykind/RelativePartialPressure': {'http://qudt.org/vocab/quantitykind/RelativeHumidity': {'b4'}}})
# iteration of all parent nested items to be done


    
print(len(q3.keys()))
print(len(q3.values()))
for k,v in q3.items():
    print(k,'\n', v,'\n')

12
12
http://qudt.org/vocab/quantitykind/ComplexPower 
 {'http://qudt.org/vocab/quantitykind/ActivePower': {'b3'}, 'http://qudt.org/vocab/quantitykind/ApparentPower': {'b3'}, 'http://qudt.org/vocab/quantitykind/ReactivePower': {'b3'}} 

http://qudt.org/vocab/quantitykind/StochasticProcess 
 {'http://qudt.org/vocab/quantitykind/Activity': {'b3'}} 

http://qudt.org/vocab/quantitykind/Angle 
 {'http://qudt.org/vocab/quantitykind/AngleOfAttack': {'b3'}, 'http://qudt.org/vocab/quantitykind/AngleOfOpticalRotation': {'b3'}, 'http://qudt.org/vocab/quantitykind/AngularDistance': {'b3'}, 'http://qudt.org/vocab/quantitykind/BevelGearPitchAngle': {'b3'}, 'http://qudt.org/vocab/quantitykind/BraggAngle': {'b3'}, 'http://qudt.org/vocab/quantitykind/PlaneAngle': {'b3'}} 

http://qudt.org/vocab/quantitykind/HeadEndPressure 
 {'http://qudt.org/vocab/quantitykind/AverageHeadEndPressure': {'b3'}} 

http://qudt.org/vocab/quantitykind/VacuumThrust 
 {'http://qudt.org/vocab/quantitykind/AverageVacuumThrust':

In [12]:
q3 = q4
for k,v in inMQ3.items():
    q3[v].update({k: 'nestetDepth_3'})


print(len(q3.keys()))
print(len(q3.values()))
for k,v in q3.items():
    print(k,'\n', v,'\n')

KeyError: 'http://qudt.org/vocab/quantitykind/ComplexPower'

In [137]:
t = {}
a =  {'irgendwas'}
b = {'anderes'}
c = {'z'}

In [138]:
t['link'] = a
print(t)

{'link': {'irgendwas'}}


In [140]:
t['link'].update(b)
print(t)

{'link': {'anderes', 'irgendwas'}}


In [141]:
t['link'].update(c)
print(t)

{'link': {'z', 'anderes', 'irgendwas'}}


In [142]:
t['newlink'] = t['link']
print(t)

{'link': {'z', 'anderes', 'irgendwas'}, 'newlink': {'z', 'anderes', 'irgendwas'}}


In [13]:
inMQ3 = {}
notinMQ3 = {}
for key,value in notinMQ2.items():
    if value in inMQ2.keys():
        inMQ3[value].update() = key
    if value not in inMQ2.keys():
        notinMQ3[key] = value

print(len(inMQ3.keys()))
print(len(notinMQ3.keys()))
for k,v in inMQ3.items():
    print(k,'\n', v,'\n')

SyntaxError: cannot assign to function call (<ipython-input-13-ecfc31d4c290>, line 5)

In [14]:
inMQ4 = {}
notinMQ4 = {}
for key,value in notinMQ3.items():
    if value in inMQ3.keys():
        inMQ4[value].update({key}) 
    if value not in inMQ3.keys():
        notinMQ4[key] = value

print(len(inMQ4.keys()))
print(len(notinMQ4.keys()))
print(inMQ4.items())

KeyError: 'http://qudt.org/vocab/quantitykind/RelativePartialPressure'

In [15]:
allQuantities = {}
for idx in quantityKind['results']['bindings']:
    allQuantities[idx['quantityKind']['value']] = {}

mainQuantities = {}
broaderQuantities = {}

for idx in quantityKind['results']['bindings']:
    if(idx['broader']['value']=='mainQ'):
        mainQuantities[idx['quantityKind']['value']] = {}
    else: 
        broaderQuantities[idx['quantityKind']['value']] = idx['broader']['value']

print(len(mainQuantities.keys()))
print(len(broaderQuantities.keys()))

for key,value in broaderQuantities.items():
    print(key,'\nbelongsToBroader\n', value, '\n')


518
343
http://qudt.org/vocab/quantitykind/AbsoluteActivity 
belongsToBroader
 http://qudt.org/vocab/quantitykind/InverseVolume 

http://qudt.org/vocab/quantitykind/AbsoluteHumidity 
belongsToBroader
 http://qudt.org/vocab/quantitykind/Density 

http://qudt.org/vocab/quantitykind/AbsorbedDose 
belongsToBroader
 http://qudt.org/vocab/quantitykind/SpecificEnergy 

http://qudt.org/vocab/quantitykind/AccelerationOfGravity 
belongsToBroader
 http://qudt.org/vocab/quantitykind/Acceleration 

http://qudt.org/vocab/quantitykind/AcceptorDensity 
belongsToBroader
 http://qudt.org/vocab/quantitykind/NumberDensity 

http://qudt.org/vocab/quantitykind/AcceptorIonizationEnergy 
belongsToBroader
 http://qudt.org/vocab/quantitykind/IonizationEnergy 

http://qudt.org/vocab/quantitykind/ActiveEnergy 
belongsToBroader
 http://qudt.org/vocab/quantitykind/Energy 

http://qudt.org/vocab/quantitykind/ActivePower 
belongsToBroader
 http://qudt.org/vocab/quantitykind/ComplexPower 

http://qudt.org/vocab/quanti

In [17]:
counter = 0
mq = mainQuantities
print(mq['http://qudt.org/vocab/quantitykind/length'])
mq.update({'http://qudt.org/vocab/quantitykind/InverseVolume':'blubb'})
print(mq['http://qudt.org/vocab/quantitykind/InverseVolume'])

KeyError: 'http://qudt.org/vocab/quantitykind/length'

In [ ]:
for k,v in broaderQuantities.items():
    mq[v].update({v: k, 'nestedBroader':1})
    counter = counter + 1

print(counter)  

In [208]:
print(mq['http://qudt.org/vocab/quantitykind/InverseVolume'])

http://qudt.org/vocab/quantitykind/NumberDensity
